# Lending Club - Case Study

EDA to understand how consumer attributes and loan attributes influence the tendency to default

Steps we are going to follow for EDA:
1. Understand the dataset 
2. Clean and filter the data 
3. Data Analysis (univariate, segmented univariate, bivariate analysis)
4. Draw conclusions

| Dataset | Description | 
| --- | --- | 
| [Loan Data Set](https://cdn.upgrad.com/UpGrad/temp/3ba74fb7-bd88-4854-8597-1c225a5aed99/loan.zip) | This contains the complete loan data for all loans issued through the time period 2007-2011. | 
| [Data Dictionary](https://cdn.upgrad.com/UpGrad/temp/af860da6-f838-47d6-ad97-551022550ee4/Data_Dictionary.xlsx) | This dataset describes the meaning of the variables mentioned in the Loan Data Set. | 

## Step 0: Pre-requisites

Importing all the required libraries for our EDA

In [1]:
# Built-in libraries
import os
import re
import ssl
from datetime import datetime as dt
from io import BytesIO, StringIO
from urllib.request import urlopen
from zipfile import ZipFile

# Third-party libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns

Code to handle SSL configuration

In [2]:
if not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context

Get the contents of the provided zip file URL for the loan dataset in a dictionary

In [3]:
# loan_data_set_url = r'https://cdn.upgrad.com/UpGrad/temp/3ba74fb7-bd88-4854-8597-1c225a5aed99/loan.zip'
loan_data_set_url = r'https://raw.githubusercontent.com/rahul-nanwani/lending-club/master/datasets/loan.zip'
loan_data_set = {}
with urlopen(loan_data_set_url) as resp:
    with ZipFile(BytesIO(resp.read()), 'r') as zf:
        for filename in zf.namelist():
            loan_data_set[filename] = zf.open(filename).read().decode('utf-8')

Check the files found from the zip file

In [4]:
list(loan_data_set.keys())

['loan.csv']

Use the `loan.csv` file to create a dataframe

In [5]:
loan_data_set = StringIO(loan_data_set['loan.csv'])
df = pd.read_csv(loan_data_set)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Step 1: Understand the dataset 

It is important to know the count of the rows and columns of the dataset we are using for our EDA 

In [6]:
df.shape

(39717, 111)

It is also important to know how the rows look like 

In [7]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [8]:
df.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
39712,92187,92174,2500,2500,1075.0,36 months,8.07%,78.42,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39713,90665,90607,8500,8500,875.0,36 months,10.28%,275.38,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39714,90395,90390,5000,5000,1325.0,36 months,8.07%,156.84,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39715,90376,89243,5000,5000,650.0,36 months,7.43%,155.38,A,A2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39716,87023,86999,7500,7500,800.0,36 months,13.75%,255.43,E,E2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


Now we glance through the [Data Dictionary](https://cdn.upgrad.com/UpGrad/temp/af860da6-f838-47d6-ad97-551022550ee4/Data_Dictionary.xlsx) provided for the loan dataset.

Our target variable is `loan_status`

## Step 2. Clean and filter the data 

It is important that we have clean and good quality data for the analysis. We need to ensure that the data we are using is not based on assumptions, so it is good that we drop the highly missing data. 

### Drop columns with high missing data ratio

Check the columns with the highest missing data ratio

In [10]:
df.isna().mean().sort_values(ascending=False).head(10)

verification_status_joint    1.0
annual_inc_joint             1.0
mo_sin_old_rev_tl_op         1.0
mo_sin_old_il_acct           1.0
bc_util                      1.0
bc_open_to_buy               1.0
avg_cur_bal                  1.0
acc_open_past_24mths         1.0
inq_last_12m                 1.0
total_cu_tl                  1.0
dtype: float64

Let us remove the columns without any data and recheck the ratio

In [11]:
df = df.loc[:, df.isna().mean()<1]

In [12]:
df.isna().mean().sort_values(ascending=False).head(10)

next_pymnt_d                  0.971297
mths_since_last_record        0.929854
mths_since_last_delinq        0.646625
desc                          0.325805
emp_title                     0.061913
emp_length                    0.027066
pub_rec_bankruptcies          0.017549
last_pymnt_d                  0.001788
chargeoff_within_12_mths      0.001410
collections_12_mths_ex_med    0.001410
dtype: float64

Let us also eliminate the columns with more than or equal to 30% of missing data, since everything there will not be of any use to us 

In [13]:
df = df.loc[:, df.isna().mean()<0.3]

### Drop columns with just 1 unique value

Let us now check for the count of unique values in a column. Columns with just one unqiue value will not benefit us so we can drop those

In [14]:
df.nunique().sort_values(ascending=True).head(10)

tax_liens                     1
delinq_amnt                   1
chargeoff_within_12_mths      1
acc_now_delinq                1
application_type              1
policy_code                   1
collections_12_mths_ex_med    1
initial_list_status           1
pymnt_plan                    1
term                          2
dtype: int64

In [15]:
df = df.loc[:, df.nunique()!=1]

In [16]:
df.shape

(39717, 44)

### Drop row duplicates

Now we need to check if there are any duplicates

In [17]:
df[df.duplicated()]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies


Let us see what columns we are left with along with their data type

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 44 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       39717 non-null  int64  
 1   member_id                39717 non-null  int64  
 2   loan_amnt                39717 non-null  int64  
 3   funded_amnt              39717 non-null  int64  
 4   funded_amnt_inv          39717 non-null  float64
 5   term                     39717 non-null  object 
 6   int_rate                 39717 non-null  object 
 7   installment              39717 non-null  float64
 8   grade                    39717 non-null  object 
 9   sub_grade                39717 non-null  object 
 10  emp_title                37258 non-null  object 
 11  emp_length               38642 non-null  object 
 12  home_ownership           39717 non-null  object 
 13  annual_inc               39717 non-null  float64
 14  verification_status   

### Load data dictionary

It would be handy to have a dataframe with the descriptions of the columns we have left. 

*Since I had opened the sheet locally I can tell that we will only require `LoanStats` sheet.*

In [19]:
# data_dict_url = r'https://cdn.upgrad.com/UpGrad/temp/af860da6-f838-47d6-ad97-551022550ee4/Data_Dictionary.xlsx'
data_dict_url = r'https://raw.githubusercontent.com/rahul-nanwani/lending-club/master/datasets/Data_Dictionary.xlsx'
data_dict = pd.read_excel(data_dict_url, sheet_name='LoanStats')

In [20]:
data_dict.head()

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...


In [21]:
data_dict.columns = ['Name', 'Description']

In [22]:
data_dict = data_dict[data_dict['Name'].isin(df.columns)]
data_dict.reset_index(inplace=True, drop=True)

Now we have a clean data dictionary we can refer to 

In [23]:
data_dict

,Name,Description
0,addr_state,The state provided by the borrower in the loan...
1,annual_inc,The self-reported annual income provided by th...
2,collection_recovery_fee,post charge off collection fee
3,delinq_2yrs,The number of 30+ days past-due incidences of ...
4,dti,A ratio calculated using the borrower’s total ...
5,earliest_cr_line,The month the borrower's earliest reported cre...
6,emp_length,Employment length in years. Possible values ar...
7,emp_title,The job title supplied by the Borrower when ap...
8,funded_amnt,The total amount committed to that loan at tha...
9,funded_amnt_inv,The total amount committed by investors for th...


### Drop irrelevant columns 

Now we can drop the columns that won't help us in our analysis. We can drop the behavioural variables that are generated after the loan approved or are not available at the time of loan application. We can also drop irrelevant demographical variables. 


In [24]:
behavioural_columns = [
    'collection_recovery_fee',
    'delinq_2yrs',
    'earliest_cr_line',
    'inq_last_6mths',
    'last_credit_pull_d',
    'last_pymnt_amnt',
    'last_pymnt_d',
    'open_acc',
    'out_prncp',
    'out_prncp_inv',
    'pub_rec',
    'recoveries',
    'revol_bal',
    'revol_util',
    'total_acc',
    'total_pymnt',
    'total_pymnt_inv',
    'total_rec_int',
    'total_rec_late_fee',
    'total_rec_prncp'
]
demographical_columns = [
    'addr_state', 
    'emp_title', 
    'id', 
    'member_id', 
    'title', 
    'url', 
    'zip_code'
]

In [25]:
df.drop(behavioural_columns+demographical_columns, axis=1, inplace=True)

### Drop irrelevant rows

We don't need rows which are on-going loans, as we cannot say if it is a defaulted loan or has paid-off. We can drop these rows.

In [26]:
df = df[df['loan_status']!='Current']
df.reset_index(inplace=True, drop=True)

### Standardise data 

Now let us go through each object column to check its actual data and clean if required 

In [27]:
df.select_dtypes(include=[object]).head()

,term,int_rate,grade,sub_grade,emp_length,home_ownership,verification_status,issue_d,loan_status,purpose
0,36 months,10.65%,B,B2,10+ years,RENT,Verified,Dec-11,Fully Paid,credit_card
1,60 months,15.27%,C,C4,< 1 year,RENT,Source Verified,Dec-11,Charged Off,car
2,36 months,15.96%,C,C5,10+ years,RENT,Not Verified,Dec-11,Fully Paid,small_business
3,36 months,13.49%,C,C1,10+ years,RENT,Source Verified,Dec-11,Fully Paid,other
4,36 months,7.90%,A,A4,3 years,RENT,Source Verified,Dec-11,Fully Paid,wedding


- Term has only 2 unique values i.e. 36 months and 60 months we can remove the months suffix later on if required. 
- For now we can clean int_rate, emp_length, and issue_d as required

In [28]:
df['int_rate'] = df['int_rate'].dropna().apply(lambda x: float(re.sub(r'[^.\d]', '', x)))

In [29]:
df['emp_length'].unique()

array(['10+ years', '< 1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '1 year', '6 years', '2 years', '7 years',
       nan], dtype=object)

Let us assume that \< 1 year as 0 and 10+ years as 10
We will extract the first two characters and typecast into int except for \< 1 year

In [30]:
df['emp_length'] = df['emp_length'].dropna().apply(lambda x: 0 if x == '< 1 year' else int(re.sub(r'\D', '', x)))

In [31]:
df['issue_d'] = df['issue_d'].dropna().apply(lambda x: dt.strptime(x, '%b-%y'))

We need only month for our analysis, let us keep only month and set its column type as category. 

In [32]:
df['issue_d'] = df['issue_d'].dropna().apply(lambda x: x.strftime('%m'))

In [33]:
df['issue_d'] = df['issue_d'].astype('category')

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38577 entries, 0 to 38576
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   loan_amnt             38577 non-null  int64   
 1   funded_amnt           38577 non-null  int64   
 2   funded_amnt_inv       38577 non-null  float64 
 3   term                  38577 non-null  object  
 4   int_rate              38577 non-null  float64 
 5   installment           38577 non-null  float64 
 6   grade                 38577 non-null  object  
 7   sub_grade             38577 non-null  object  
 8   emp_length            37544 non-null  float64 
 9   home_ownership        38577 non-null  object  
 10  annual_inc            38577 non-null  float64 
 11  verification_status   38577 non-null  object  
 12  issue_d               38577 non-null  category
 13  loan_status           38577 non-null  object  
 14  purpose               38577 non-null  object  
 15  dt

All the remaining columns of type object are categorical variables here 

In [35]:
categorical_columns = list(df.select_dtypes(include=[object]).columns)
df[categorical_columns] = df[categorical_columns].astype('category')

In [36]:
data_dict = data_dict[data_dict['Name'].isin(df.columns)]
data_dict.reset_index(inplace=True, drop=True)

## Step 3. Data Analysis

In [37]:
data_dict

,Name,Description
0,annual_inc,The self-reported annual income provided by th...
1,dti,A ratio calculated using the borrower’s total ...
2,emp_length,Employment length in years. Possible values ar...
3,funded_amnt,The total amount committed to that loan at tha...
4,funded_amnt_inv,The total amount committed by investors for th...
5,grade,LC assigned loan grade
6,home_ownership,The home ownership status provided by the borr...
7,installment,The monthly payment owed by the borrower if th...
8,int_rate,Interest Rate on the loan
9,issue_d,The month which the loan was funded


## Step 4. Draw Conclusion

After completing our analysis we have observed that,

| Category | Least likely to default | Most likely to default | 
| --- | --- | --- |
| xxx | xxx (x%) | xxx (x%) | 
| xxx | xxx (x%) | xxx (x%) | 

*Note: the table only shows the most and the least likely to default from the particular category*